# Padova Grand Tour - Queries

This notebook will provide ten insightful queries about our data. Get started by starting GraphDB with `docker compose up --force-recreate`.


Sparql wrapper (copy and pasted from the Individual Project's notebooks):

In [79]:

from SPARQLWrapper import SPARQLWrapper, JSON

# Taken from `data/ttlData/ontology.ttl`
prefixString = """
PREFIX : <https://padovagrandtour.github.io/entitites#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX pgt: <https://padovagrandtour.github.io/entitites#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX xml: <http://www.w3.org/XML/1998/namespace> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    # Our local endpoint, the repo name is defined in `data/graphdb-repo.ttl` 
    sparql = SPARQLWrapper("http://localhost:7210/repositories/pgt") 
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            array.append(app)
        print(len(array)) # Print the array length
        return array

    except Exception as e :
        print("The operation failed", e)
    

In [80]:
# Improved version that also handles prefixes
mappedPrefixes = {}
for prefix in prefixString.split('\n'):
    if('PREFIX' in prefix):
        prefix = prefix.replace('PREFIX', '', 1).split(':', 1)
        mappedPrefixes[prefix[0].strip()] = prefix[1].strip().replace('<','').replace('>','')

def query(queryString):
    queryResults = run_query(queryString)
    for queryIndex, queryResultRaw in enumerate(queryResults):
        for subQueryIndex, subQueryResultRaw in enumerate(queryResultRaw):
            queryResult = queryResultRaw[subQueryIndex][1]
            for prefixName, prefixValue in mappedPrefixes.items():
                queryResult = queryResult.replace(prefixValue, prefixName + ':')

            queryResults[queryIndex][subQueryIndex] = (queryResultRaw[subQueryIndex][0], queryResult)

    return queryResults


Check if everything works:

In [81]:
query("""
select distinct * {
    ?s ?p ?o
}
LIMIT 3
""")


3


[[('s', 'rdf:type'), ('p', 'rdf:type'), ('o', 'rdf:Property')],
 [('s', 'rdfs:subPropertyOf'), ('p', 'rdf:type'), ('o', 'rdf:Property')],
 [('s', 'rdfs:subPropertyOf'),
  ('p', 'rdf:type'),
  ('o', 'owl:TransitiveProperty')]]